In [39]:
#load fetch_ml is depreciated now pass through fetch fetch_openml
from sklearn.datasets import fetch_openml
import numpy as np
import pandas as pd
#load drive lib
#scoring lib
from mlxtend.evaluate import bias_variance_decomp

In [2]:
#there is a bug on os x with python version > 3.6 with ssl certificate. 
#https://github.com/scikit-learn/scikit-learn/issues/10201
#workaround above 
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

In [10]:
#loading minst form the cloud
mnist = fetch_openml('mnist_784', version=1)


In [15]:
minstFrame = pd.DataFrame(mnist.data)
minstFrame.describe()
mnistFrameTarget = pd.DataFrame(mnist.target)

# Le dataset principal qui contient toutes les images
#print (mnist.data.shape)

# Le vecteur d'annotations associé au dataset (nombre entre 0 et 9)
#print (mnist.target.shape)
#print(mnist.data)

In [25]:
#reduction de la taille du dataset
#generation de 20k index aléatoires et compris entre 1 et 70k
# pour des raisons assez obscure le filtre par entier ne fonctionne pas en local sur le ndarray comme sur colab
# on utilise un workaround avec pandas et loc
sample = np.random.randint(70000, size=50000)
#on sample 20k vecteurs depuis data
data = minstFrame.loc[list(sample)]
target = mnistFrameTarget.loc[list(sample)]

In [26]:
#on split le dataset et le set de resulat pour l'enrtainement et le test avec un classique 80/20
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(data, target, train_size=0.8)

In [31]:
#loading de la lib knn
from sklearn import neighbors
#n_neighbours est le nombre de voisin autour du point pris pour calculer la classification du future point
#dans l'exemple concernée soit bleu soit rouge
model = neighbors.KNeighborsClassifier(n_neighbors=5)
#A column-vector y was passed when a 1d array was expected
model.fit(data.to_numpy(), target.values.ravel())

KNeighborsClassifier()

In [34]:
#test du model, il attend un tableau 
model.predict([xtest.iloc[100]])

array(['9'], dtype=object)

In [35]:
#pour visualiser le vecteur en question sous forme de tableau d'int de nuances de gris 
xtest.iloc[100]

pixel1      0.0
pixel2      0.0
pixel3      0.0
pixel4      0.0
pixel5      0.0
           ... 
pixel780    0.0
pixel781    0.0
pixel782    0.0
pixel783    0.0
pixel784    0.0
Name: 52123, Length: 784, dtype: float64

In [36]:
#check error for the current model 
error = model.score(xtest, ytest)
print(1 - error)
a = range(2, 15, 1)
print(a[2])

0.019000000000000017
4


In [ ]:
#check where the model is the most accurate by tweaking the amount of neighbours from 2 to 15
errors = []
for k in range(2, 15):
  print(k)
  newmodel = neighbors.KNeighborsClassifier(n_neighbors=k)
  newmodel.fit(xtrain, ytrain)
  errors.append((1-newmodel.score(xtest, ytest))*100)

#print plot error rate for each neighbours size

In [ ]:
print(errors)
import matplotlib.pyplot as plt
#plt.
plt.plot(range(2,15), errors, 'o-')
plt.show()


In [97]:
#take out of the loop the data for testing

sample = np.random.randint(70000, size=10000)
#it is necessary to replace 0 by 1 in x to avoid issue while scoring bias and var
data = minstFrame.loc[list(sample)].sort_index().replace(0, 1).to_numpy()
target = mnistFrameTarget["class"].astype(int).loc[list(sample)].sort_index().to_numpy()
Xtrain, xtest, Ytrain, ytest = train_test_split(data, target, train_size=0.8)

for size in range(5000, 20000, 5000):
    performanceFrame = pd.DataFrame(data={"r2": [], "loss": [], "bias": [], "var": [], "knn": [], "size": []})
    print(size)
    sample = np.random.randint(70000, size=size)
    data = minstFrame.loc[list(sample)].sort_index().replace(0,1).to_numpy()
    target = mnistFrameTarget["class"].astype(int).loc[list(sample)].sort_index().to_numpy()
    xtrain, xtest2, ytrain, ytest2 = train_test_split(data, target, train_size=0.8)
    print(xtrain)
    print(ytrain)
    for neighbours in range(2, 11, 2):
        model = neighbors.KNeighborsClassifier(n_neighbors=neighbours)
        print("neighbours", neighbours)
        #A column-vector y was passed when a 1d array was expected
      
        #loss can be mse or 0-1_loss
        #loss, avg_bias, avg_var = bias_variance_decomp(model, xtrain.to_numpy(), ytrain.values.ravel(), xtest.to_numpy(), ytest.values.ravel(), loss='0-1_loss', random_seed=123)
        loss, avg_bias, avg_var = bias_variance_decomp(model, xtrain, ytrain.ravel(), xtest, ytest, loss='mse', random_seed=123)
        #model.fit(data.to_numpy(), target.values.ravel())
        #model.fit(xtrain.to_numpy(), ytrain.values.ravel())
        model.fit(xtrain, ytrain.ravel())
        print("after fit")
        #R2 = (1-model.score(xtest.to_numpy(), ytest.values.ravel()))*100
        R2 = (1-model.score(xtest, ytest.ravel()))*100
        print("after R2", round(R2, 2))
        
        currentPerformance = pd.DataFrame([{'r2': R2, "loss": loss, "var": avg_var, "bias": avg_bias, "size": size, "knn": neighbours}])
        performanceFrame = performanceFrame.append(currentPerformance)      
    performanceFrame.to_csv("model_knn_fixedTest_performance_"+str(size)+".csv", index=False)
print(performanceFrame.describe())

5000
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
[5 5 1 ... 7 7 2]
neighbours 2
after fit
after R2 8.55
neighbours 4
after fit
after R2 7.2
neighbours 6
after fit
after R2 7.55
neighbours 8
after fit
after R2 7.2
neighbours 10
after fit
after R2 7.9
10000
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
[0 7 2 ... 5 3 8]
neighbours 2


KeyboardInterrupt: 